In [4]:
import datasets
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import BertTokenizer

max_input_length = 512
max_target_length = 128
 
lcsts_part_1=pd.read_table('../../../resources/corpus/modules/sum/lscts/DATA/PART_I.txt', header=None,
                           warn_bad_lines=True, error_bad_lines=False, sep='<[/d|/s|do|su|sh][^a].*>', encoding='utf-8') # PART I  too big, use PART II instead
lcsts_part_1=lcsts_part_1[0].dropna()
lcsts_part_1=lcsts_part_1.reset_index(drop=True)
lcsts_part_1=pd.concat([lcsts_part_1[1::2].reset_index(drop=True), lcsts_part_1[::2].reset_index(drop=True)], axis=1)
lcsts_part_1.columns=['document', 'summary']
 
lcsts_part_2=pd.read_table('../../../resources/corpus/modules/sum/lscts/DATA/PART_II.txt', header=None,
                           warn_bad_lines=True, error_bad_lines=False, sep='<[/d|/s|do|su|sh][^a].*>', encoding='utf-8')
lcsts_part_2=lcsts_part_2[0].dropna()
lcsts_part_2=lcsts_part_2.reset_index(drop=True)
lcsts_part_2=pd.concat([lcsts_part_2[1::2].reset_index(drop=True), lcsts_part_2[::2].reset_index(drop=True)], axis=1)
lcsts_part_2.columns=['document', 'summary']
 
dataset_train = Dataset.from_dict(lcsts_part_1)
dataset_valid = Dataset.from_dict(lcsts_part_2)


/tmp/ipykernel_17144/2001870770.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  lcsts_part_1=pd.read_table('../../../resources/corpus/modules/sum/lscts/DATA/PART_I.txt', header=None,
/tmp/ipykernel_17144/2001870770.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  lcsts_part_1=pd.read_table('../../../resources/corpus/modules/sum/lscts/DATA/PART_I.txt', header=None,
/tmp/ipykernel_17144/2001870770.py:9: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  lcsts_part_1=pd.read_table('../../../resources/corpus/modules/sum/lscts/DATA/PART_I.txt', header=None,
/tmp/ipykernel_17144/2001870770.py:16: ParserWarning: F

In [5]:
print(len(dataset_train))
print(len(dataset_valid))

2400591
10666


In [6]:
print(dataset_train[0])

{'document': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'summary': '修改后的立法法全文公布'}


In [8]:
d_len = [len(tokenizer.encode(s['document'])) for s in dataset_train] 
s_len = [len(tokenizer.encode(s['summary'])) for s in dataset_train] 

NameError: name 'plt' is not defined

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0") 
axes[0].set_title("Dialogue Token Length") 
axes[0].set_xlabel("Length") 
axes[0].set_ylabel("Count") 
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0") 
axes[1].set_title("Summary Token Length") 
axes[1].set_xlabel("Length") 
plt.tight_layout() 

In [ ]:
TokenModel = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(TokenModel)

In [ ]:

def preprocess_function(examples):
    inputs = [str(doc) for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    inputs = [str(doc) for doc in examples["summary"]]
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(inputs, max_length=max_target_length, truncation=True)
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets_t = dataset_train.map(preprocess_function, batched=True)
tokenized_datasets_v = dataset_valid.map(preprocess_function, batched=True)
 
tokenized_datasets = datasets.DatasetDict({"train":tokenized_datasets_t,"validation": tokenized_datasets_v})
print(tokenized_datasets)

print(tokenized_datasets['train'][0])

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download("punkt")

In [ ]:
sample_text = 

In [ ]:
from transformers import pipeline, set_seed 
pipe = pipeline("summarization", model="facebook/bart-large-cnn") 
pipe_out = pipe(sample_text) 
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))